In [58]:
import json
import pandas as pd
import numpy as np

PATH = "Prakhar/er-assignment/fs/Instabase%20Drive/files/datasets/"
FILES = {
    "foursquare_test": "foursquare_test_hard.json",
    "locu_test": "locu_test_hard.json",
    "matches": "matches_train_hard.csv",
    "foursquare_train": "foursquare_train_hard.json",
    "locu_train": "locu_train_hard.json"
}

In [93]:
fs_train = pd.read_json(ib.open(PATH + FILES["foursquare_train"]))
fs_test = pd.read_json(ib.open(PATH + FILES["foursquare_test"]))
lc_train = pd.read_json(ib.open(PATH + FILES["locu_train"]))
lc_test = pd.read_json(ib.open(PATH + FILES["locu_test"]))
matches = pd.read_csv(ib.open(PATH + FILES["matches"]))

data_list = [fs_train, fs_test, lc_train, lc_test]
for df in data_list:
    df.replace([''], [None], inplace=True)
    df['phone'].replace([r'\(|\)|\s|-'], [''], regex=True, inplace=True)

In [94]:
fs_train

,country,id,latitude,locality,longitude,name,phone,postal_code,region,street_address,website
0,United States,4f328ea619836c91c7e3714a,40.794855,New York,-73.966069,Chen Jin Diao Restaurant,2126784680,None,NY,800 Columbus Ave.,None
1,United States,4c37b5f6ae2da593a56affc5,40.760460,New York,-73.994920,West Side Steakhouse,2125644803,10036,NY,597 10th Ave.,None
2,United States,4b41060df964a52098bf25e3,40.760249,New York,-73.983746,Pearls: Chinese & Szechuan Cuisine,2125827380,10019,NY,732 7th Ave.,None
3,United States,3fd66200f964a520ece41ee3,40.708660,New York,-74.011263,Suspenders,2127325005,10006,NY,111 Broadway,http://www.suspendersnyc.com
4,United States,52064aab11d284f64d088329,40.718105,New York,-73.996096,Grand Century Cafe,None,None,NY,None,None
5,United States,40ede000f964a5203c0a1fe3,40.785676,New York,-73.984509,Boat Basin Café,2124965542,10024,NY,390 W. 79th St.,None
6,United States,49c406a8f964a5208d561fe3,40.746279,New York,-73.997950,Brooklyn Bagel & Coffee Co.,2129242824,10001,NY,286 8th Ave.,http://www.brooklynbagelandcoffeecompany.com/o...
7,United States,5177d30c498e9b657328e30f,40.793718,New York,-73.967123,Coffee Cart - 97th & Columbus,None,10025,NY,W. 97th St.,None
8,United States,4edfa9edd5fbc1bbdd9b43c8,40.784835,New York,-73.947574,Johnny Bananas,None,10128,NY,231 E. 96th St. Apt 6r,None
9,United States,51cb0e7d498e5a55996ed389,40.820919,New York,-73.936461,Jimbos,2124918680,10037,NY,535 Malcolm X Blvd.,None


0      (212) 777-7131
1                None
2                None
3                None
4                None
5      (212) 262-9263
6      (212) 475-9540
7      (212) 245-0086
8      (212) 245-0086
9                None
10     (212) 226-0559
11     (212) 226-0559
12     (212) 432-3634
13     (212) 317-1999
14     (212) 695-4080
15               None
16     (212) 679-4555
17     (212) 414-3003
18     (212) 423-0599
19     (212) 661-3400
20     (212) 517-3800
21               None
22               None
23     (212) 732-5555
24               None
25     (212) 577-7550
26     (212) 249-7100
27     (212) 777-4650
28     (212) 777-4650
29               None
            ...      
370              None
371    (212) 473-4493
372    (212) 460-5300
373    (212) 736-1575
374              None
375    (212) 223-1206
376              None
377              None
378    (212) 543-2111
379    (212) 691-4570
380              None
381    (212) 777-2808
382    (212) 791-8981
383              None
384       